In [2]:
import glob
from pathlib import Path
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Utils

In [ ]:
def lower_text(text):
    return text.lower()
def hard_remove_spaces(text):
    return " ".join(text.split())

lowerText = lambda x: lower_text(x)
hardRemoveSpaces = lambda x: hard_remove_spaces(x)

# Aggregate

In [ ]:
file_ = glob.glob("../sites_data/*")[0]
df=pd.read_csv(file_,index_col=None)#.drop('Unnamed: 0', 1)
df['source'] = Path(file_).stem.split('-')[0]
print(Path(file_).stem)

for file_ in glob.glob("../sites_data/*")[1:]:
    print(Path(file_).stem)
    df_file = pd.read_csv(file_,index_col=None)
    df_file['source'] = Path(file_).stem.split('-')[0]
    df=pd.concat([df,df_file],axis=0).reset_index(drop=True)

In [ ]:
print(df.columns)

In [ ]:
selected_columns = ['url', 'title', 'salary', 'experiment', 'degree', 'so_luong',
       'industry', 'location', 'level', 'jd', 'benefit', 'requirement',
       'source', ]

df = df[selected_columns]

print(f'Number of jobs post: {df.shape[0]}')
df['source'].value_counts()

# Language detect

In [ ]:
df['language'] = pd.Series(0, index=df.index)

from langdetect import detect
for i in range(df.shape[0]):
    # REINITIALIZE THE API
    try:        
        df['language'][i] = detect(str(df.requirement[i]))
    except:
        continue
    if i%10000==0:         print(i)

In [ ]:
df['language'].value_counts().to_frame()

## `vi` and `en`

In [ ]:
df = df.loc[(df.language == 'vi') | (df.language == 'en')].reset_index(drop=True)#.language.value_counts()

In [ ]:
df['source'].value_counts().to_frame()

In [ ]:
df[~df['degree'].isnull()]['source'].value_counts()#.reset_index(drop=True).astype(str)


In [ ]:
df['degree'].value_counts()

# blue_or_white
- Clean up degree
- Blue:
    - Site that blue in nature
        - tuyencongnhan
    - `degree` is in `blue_degree_list`

In [ ]:
df['blue_or_white'] = 'white'

## Site that blue in nature
- tuyencongnhan

In [ ]:
df.loc[df.source=='tuyencongnhan','blue_or_white'] = 'blue'

## By degree

### Clean up degree

In [ ]:
df.loc[~df['degree'].isnull(),'degree'] = df.loc[~df['degree'].isnull(),'degree'].apply(hardRemoveSpaces)
df.loc[~df['degree'].isnull(),'degree'] = df.loc[~df['degree'].isnull(),'degree'].apply(lowerText)

In [ ]:
df['degree'].value_counts()
# df

### Assigning blue tag

In [ ]:
blue_degree_list = ['trung cấp', 'trung học',  'không yêu cầu',
                    'chứng chỉ', 'trung học phổ thông','trung cấp - nghề', 
                    'chứng chỉ chuyên ngành', 'lao động phổ thông', 'chứng chỉ - sơ cấp nghề',
                    ]
df.loc[df['degree'].isin(blue_degree_list),'blue_or_white'] = 'blue'

### English jobs are blue

In [ ]:
df.loc[df['blue_or_white']=='blue','language'].value_counts()

# df.loc[df.language=='en','blue_or_white'] = 'white'

In [ ]:
df.blue_or_white.value_counts()

In [3]:
df = pd.read_csv('aggregated-2021-11-06.csv')

# Data statistics

In [4]:
ff = df.source.value_counts().to_frame()
ff

,source
careerBuilder,23922
topCV,23603
jobsGo,18397
sieuthivieclam,13964
vietnamworks,7947
vieclam24h,1102
tuyencongnhan,1082
timviecnhanh,980


## Sites

# Job requirement

## Clean up requirement

### requirement to list of requirements

In [5]:
import re
def string_to_list(text):
    string_list = re.split('\n|\t|•|-',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

In [6]:
df['requirement_list'] = df.requirement.apply(stringToList)

### Clean requirement_list

In [7]:
def scrub_words(text):
    """Basic cleaning of texts."""
    
    # remove html markup
    text=re.sub("(<.*?>)"," ",text)
    
    #remove non-ascii and digits
    text=re.sub("(\\W|\\d)"," ",text)
    
    #remove whitespace
    text = " ".join(text.split())

    text = text.lower()

    return text

def scrub_words_list(text_list):
    cleaned_text_list = []
    for text in text_list:
        cleaned_text_list.append(scrub_words(text))
    return list(filter(None, cleaned_text_list))

scrubWords = lambda x: scrub_words(x)
scrubWordsList = lambda x: scrub_words_list(x)

In [8]:
df['requirement_list_cleaned'] = df.requirement_list.apply(scrubWordsList)

## Tokenize requirements

### Vietnamese

In [9]:
from underthesea import word_tokenize #as word_tokenize_vi

def word_tokenize_from_text_vi(text):
    return word_tokenize(text)

def word_tokenize_from_text_vi_2(text):
    return word_tokenize(text,format="text").split()

def word_tokenize_from_list_vi(list_of_text):
    words = []
    for text in list_of_text:
        words += word_tokenize_from_text_vi(text)
    return words

def word_tokenize_from_list_vi_2(list_of_text):
    words = []
    for text in list_of_text:
        words+=word_tokenize_from_text_vi_2(text)
    return words

tokenizeListVi = lambda x:word_tokenize_from_list_vi(x)
tokenizeListVi2 = lambda x:word_tokenize_from_list_vi_2(x)

In [10]:
df['tokenized_requirement'] = np.NaN

In [ ]:
df.loc[df.language == 'vi','tokenized_requirement'] = df.loc[df.language == 'vi','requirement_list_cleaned'].apply(tokenizeListVi)

In [12]:
df.loc[df.language == 'vi','tokenized_requirement_2'] = df.loc[df.language == 'vi','requirement_list_cleaned'].apply(tokenizeListVi2)

### English

Now tokenizing English is harsh:
- remove step words
- stem
- so tokenizing basically break string into words
May consider other methods

#### Clean english a bit more

In [13]:
import nltk
from nltk.stem import PorterStemmer

In [14]:
def remove_stop_from_string(text):
    stopwords = nltk.corpus.stopwords.words('english')
    return ' '.join([word for word in text.split() if word not in stopwords])
    
def remove_stop_from_string_for_list(list_of_text):
    cleaned_list = []
    for text in list_of_text:
        cleaned_list.append(remove_stop_from_string(text))
    return cleaned_list

removeStopforListEn = lambda x: remove_stop_from_string_for_list(x)

def porter_stemm(text):
    porter_stemmer=PorterStemmer()
    stemmed_words=[porter_stemmer.stem(word=word) for word in text.split()]
# stemmed_words = set(stemmed_words)
    return ' '.join(stemmed_words)

def porter_stemm_for_list(list_of_text):
    stemmed_list = []
    for text in list_of_text:
        stemmed_list.append(porter_stemm(text))
    return stemmed_list

porterStemmingListEn = lambda x:porter_stemm_for_list(x)

In [15]:
df.loc[df.language == 'en','requirement_list_cleaned'] = df.loc[df.language == 'en','requirement_list_cleaned'].apply(removeStopforListEn)

df.loc[df.language == 'en','requirement_list_cleaned'] = df.loc[df.language == 'en','requirement_list_cleaned'].apply(porterStemmingListEn)

In [16]:
from nltk.tokenize import word_tokenize

def word_tokenize_from_text_en(text):
    return word_tokenize(text)

def word_tokenize_from_list_en(list_of_text):
    words = []
    for text in list_of_text:
        words += word_tokenize_from_text_en(text)
    return words

tokenizeListEn = lambda x:word_tokenize_from_list_en(x)

In [17]:
df.loc[df.language == 'en','tokenized_requirement'] = df.loc[df.language == 'en','requirement_list_cleaned'].apply(tokenizeListEn)

## Analyze requirement keywords

In [ ]:
df

In [ ]:
import itertools
from collections import Counter

In [ ]:
query_column = 'blue_or_white'
query_word = 'blue'
querydf = df.loc[df[query_column] == query_word]

keywords = list(itertools.chain(*querydf.loc[:,'tokenized_requirement']))
counts = Counter(keywords)



In [ ]:
query_words_degree = ['trung cấp', 'trung học',  
'không yêu cầu',
                    'chứng chỉ', 'trung học phổ thông','trung cấp - nghề', 
                    'chứng chỉ chuyên ngành', 'lao động phổ thông', 'chứng chỉ - sơ cấp nghề',
                    ]

In [ ]:
keywords = {
    'kn1-giaotiep': ['giao tiếp', 'đàm phán', 'thuyết phục', 'thuyết trình', 'thân thiện', 'rõ ràng',
                    'trình bày', 'tác phong', 'lắng nghe', 'truyền đạt', 'cởi mở', 'khéo léo',
                    'tương tác', 'tiếp thu', 'phản biện', 'hoạt ngôn', 'hòa nhã', 'thương thảo',
                    'ứng xử', 'thương thuyết', 'tự nhiên', 'trôi chảy', 'rành mạch',
                    'linh hoạt', 'hợp tác', 'ứng biến'],
    'kn2-xahoi': ['nhóm', 'đàm phán', 'thuyết phục', 'thuyết trình', 'chăm sóc', 'khách hàng',
                    'lãnh đạo', 'trình bày', 'thương lượng', 'phản biện'     ],
    'kn3': [],
    'kn4': [],
    'kn5': [],
    'kn6': [],
    'kn7': []
}

In [ ]:
keywords2 = {
    'kn1': ['giao_tiếp', 'đàm_phán', 'thuyết_phục', 'thuyết_trình', 'thân_thiện', 'rõ_ràng', 'trình_bày', 
                    'tác_phong','lắng_nghe', 'truyền_đạt', 'cởi_mở', 'khéo_léo', 'tương_tác', 'tiếp_thu', 
                    'phản_biện', 'hoạt_ngôn','hòa_nhã', 'thương_thảo', 'giao_tiếp_ứng_xử', 'thương_thuyết', 
                    'tự_nhiên','trôi_chảy','rành_mạch', 'giao_tiếp_linh_hoạt', 'giao_tiếp_hợp_tác', 
                    'giao_tiếp_ứng_biến'
                    ],
    'kn2': ['làm_việc nhóm', 'đàm_phán', 'thuyết_phục', 'thuyết_trình', 'chăm_sóc', 
                    'khách_hàng', 'lãnh_đạo', 'trình_bày', 'thương_lượng', 'phản_biện'],
    'kn3': ['trung_thực', 'năng_động', 'nhanh_nhẹn', 'nhiệt_tình', 'cẩn_thận', 'tinh_thần trách_nhiệm', 
            'chủ_động', 'linh_hoạt', 'thái_độ', 'tự_tin', 'hòa_đồng', 
            'tác_phong chuyên_nghiệp'],
    'kn4': ['excel', 'tin_học văn_phòng', 'word', 'vi_tính', 'vi_tính văn_phòng', 'ms office', 'microsoft office', 
                'microsoft word', 'microsoft excel'],
    'kn5': ['phân_tích', 'giải_quyết vấn_đề', 'nhạy_bén', 'tư_duy logic', 'quyết_đoán', 'xử_lý vấn_đề', 
                'problem solving', 'ứng_biến', 'lập_luận', 'xử_lý_giải_quyết', 'nhìn_nhận', 'vấn_đề', 'đào_sâu'],

    'kn6': ['làm_việc', 'độc_lập', 'chịu', 'được', 'áp_lực', 'công_việc', 'lãnh_đạo', 
                'quản_lý', 'thời_gian', 'kỹ_năng lập_kế_hoạch'],
    'kn7': ['ngoại_ngữ','tiếng anh giao_tiếp','tiếng anh giao_tiếp','tiếng anh tốt',
                'tiếng anh cơ_bản','tiếng anh thành_thạo','giao_tiếp tiếng anh cơ_bản']
}

In [ ]:
def count_keyword_frequency(df,query_column, query_value, path, save=False, ):
    counts_df = pd.DataFrame()
    for key in list(keywords2.keys()):
        keywords = keywords2[key]
        querydf = df.loc[df[query_column] == query_value]
        querydf = querydf.loc[~df['tokenized_requirement_2'].isnull()]
        
        words = list(itertools.chain(*querydf.loc[:,'tokenized_requirement_2']))
        counts = Counter(words)

        counts_df_ = pd.DataFrame.from_dict(counts, orient='index').reset_index()
        counts_df_.columns = ['keywords','count']

        counts_df_ = counts_df_.loc[counts_df_['keywords'].isin(keywords)]
        counts_df_ = counts_df_.sort_values('count',ascending=False).reset_index(drop=True)

        counts_df = pd.concat([counts_df,counts_df_],axis=1)
        # counts_df.to_excel(f"counters/{query_column} - {query_value}.xlsx", engine='xlsxwriter')

        if save == True:
            writer = pd.ExcelWriter(path, engine='openpyxl', if_sheet_exists='replace', mode='a')
            counts_df.to_excel(writer, sheet_name = f"{query_value}",index=False)
            writer.save()
            writer.close()

        
    return counts_df

In [ ]:
for d in query_words_degree:
    count_keyword_frequency(df,'degree',d,path = 'counters/degree.xlsx', save=True)



In [ ]:
def cleaner(text):
    texts = text.split()
    l = []
    for t in texts:
        l.append(re.sub('_',' ',t))
    return l

def cleaner_2(text):
    texts = text.split()
    return texts

In [ ]:
t1 = 'giao_tiếp đàm_phán thuyết_phục thuyết_trình thân_thiện rõ_ràng trình_bày tác_phong lắng_nghe truyền_đạt cởi_mở \
    khéo_léo tương_tác tiếp_thu phản_biện hoạt_ngôn hòa_nhã thương_thảo giao_tiếp_ứng_xử thương_thuyết tự_nhiên \
        trôi_chảy rành_mạch giao_tiếp_linh_hoạt giao_tiếp_hợp_tác giao_tiếp_ứng_biến'

t2 = 'làm_việc nhóm đàm_phán thuyết_phục thuyết_trình chăm_sóc khách_hàng lãnh_đạo trình_bày thương_lượng phản_biện'

t3 = 'trung_thực năng_động nhanh_nhẹn nhiệt_tình cẩn_thận tinh_thần trách_nhiệm chủ_động linh_hoạt thái_độ tự_tin \
    hòa_đồng tác_phong chuyên_nghiệp'
t4 = 'excel tin_học văn_phòng word vi_tính vi_tính văn_phòng ms office microsoft office microsoft word microsoft excel'

t5 = 'phân_tích giải_quyết vấn_đề nhạy_bén tư_duy logic quyết_đoán xử_lý vấn_đề problem solving ứng_biến \
    lập_luận xử_lý_giải_quyết nhìn_nhận vấn_đề đào_sâu'


t6 = 'phân_tích giải_quyết vấn_đề nhạy_bén tư_duy logic quyết_đoán xử_lý vấn_đề problem solving ứng_biến \
    lập_luận xử_lý_giải_quyết nhìn_nhận vấn_đề đào_sâu'

t7 = 'làm_việc độc_lập chịu được áp_lực công_việc lãnh_đạo quản_lý thời_gian kỹ_năng lập_kế_hoạch'

t8 = 'ngoại_ngữ tiếng anh giao_tiếp tiếng anh giao_tiếp tiếng anh tốt giao_tiếp tiếng anh tiếng anh cơ_bản \
    tiếng anh thành_thạo giao_tiếp tiếng anh cơ_bản'


In [18]:
# counts_df.to_excel("degree_keywords.xlsx", engine='xlsxwriter')
df.columns

Index(['url', 'title', 'salary', 'experiment', 'degree', 'so_luong',
       'industry', 'location', 'level', 'jd', 'benefit', 'requirement',
       'source', 'language', 'blue_or_white', 'requirement_list',
       'requirement_list_cleaned', 'tokenized_requirement',
       'tokenized_requirement_2'],
      dtype='object')

# Statistics

In [49]:
df.source.value_counts()
# df.loc[df.language=='en'].source.value_counts()
# df.shape

careerBuilder     23922
topCV             23603
jobsGo            18397
sieuthivieclam    13964
vietnamworks       7947
vieclam24h         1102
tuyencongnhan      1082
timviecnhanh        980
Name: source, dtype: int64

In [55]:
['en']*8

['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en']

In [139]:
import plotly.graph_objects as go
ids = ['total','vi','en'] + list('vi - '+df.source.unique()) + list('en - '+df.source.unique()) 
labels = ['total','vi','en'] + list(''+df.source.unique()) +list(''+df.source.unique()) 
parents = ['','total','total'] + ['vi']*8 + ['en']*8

values = [df.shape[0]] + [df.loc[df.language=='vi'].shape[0]] + [df.loc[df.language=='en'].shape[0]]

for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.language == 'vi')].shape[0])

for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.language == 'en')].shape[0])

fig =go.Figure(go.Sunburst(
 ids=ids,
  labels= labels,
  parents=parents,
  values = values,
  branchvalues="total",
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()

In [151]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(1, 2, specs=[[{"type": "domain"}, {"type": "domain"}]],)

ids = ['total','vi','en'] + list('vi - '+df.source.unique()) + list('en - '+df.source.unique()) 
labels = [f'Total {df.shape[0]} jobs','vi','en'] + list(''+df.source.unique()) +list(''+df.source.unique()) 
parents = ['','total','total'] + ['vi']*8 + ['en']*8

values = [df.shape[0]] + [df.loc[df.language=='vi'].shape[0]] + [df.loc[df.language=='en'].shape[0]]

for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.language == 'vi')].shape[0])

for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.language == 'en')].shape[0])

fig.add_trace(go.Sunburst(ids = ids,
                            labels = labels,
                            parents=parents,
                            values = values,
                             branchvalues="total",
                            ), 1, 1)
ids = ['total' ]+ list(df.source.unique()) + list(df.source.unique()+' - en') + list(df.source.unique()+' - vi') 
labels = [f'Total {df.shape[0]} jobs' ]+ list(df.source.unique()) + ['en']*8 + ['vi']*8
parents = [""] + ["total"]*8 + list(df.source.unique()) *2

values = [df.shape[0]]
for source in df.source.unique():
    values.append(df.loc[df.source==source].shape[0])
for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.language == 'en')].shape[0])
for source in df.source.unique():  
    values.append(df.loc[(df.source==source) & (df.language == 'vi')].shape[0])

fig.add_trace(go.Sunburst(ids = ids,
                            labels = labels,
                            parents=parents,
                            values = values,
                             branchvalues="total",
                            ), 1, 2)



fig.update_layout(
    grid= dict(columns=2, rows=1),
    margin = dict(t=1, l=30, r=30, b=1),
    width=1200, height=600
)

fig.show()

In [157]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(1, 2, specs=[[{"type": "domain"}, {"type": "domain"}]],
                     )

ids = ['total','blue','white'] + list('blue - '+df.source.unique()) + list('white - '+df.source.unique()) 
labels = [f'Total {df.shape[0]} jobs','thợ','thầy'] + list(''+df.source.unique()) +list(''+df.source.unique()) 
parents = ['','total','total'] + ['blue']*8 + ['white']*8

values = [df.shape[0]] + [df.loc[df.blue_or_white=='blue'].shape[0]] + [df.loc[df.blue_or_white=='white'].shape[0]]

for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.blue_or_white == 'blue')].shape[0])

for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.blue_or_white == 'white')].shape[0])

fig.add_trace(go.Sunburst(ids = ids,
                            labels = labels,
                            parents=parents,
                            values = values,
                             branchvalues="total",
                            ), 1, 1)
ids = ['total' ]+ list(df.source.unique()) + list(df.source.unique()+' - en') + list(df.source.unique()+' - vi') 
labels = [f'Total {df.shape[0]} jobs' ]+ list(df.source.unique()) + ['thợ']*8 + ['thầy']*8
parents = [""] + ["total"]*8 + list(df.source.unique()) *2

values = [df.shape[0]]
for source in df.source.unique():
    values.append(df.loc[df.source==source].shape[0])
for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.blue_or_white == 'blue')].shape[0])
for source in df.source.unique():  
    values.append(df.loc[(df.source==source) & (df.blue_or_white == 'white')].shape[0])

fig.add_trace(go.Sunburst(ids = ids,
                            labels = labels,
                            parents=parents,
                            values = values,
                             branchvalues="total",
                            ), 1, 2)



fig.update_layout(
    grid= dict(columns=2, rows=1),
    margin = dict(t=1, l=30, r=30, b=1),
    width=1200, height=600,
)
# fig.set_tit
fig.show()

In [ ]:
import datetime
date_object = datetime.date.today()
df.to_csv(f'aggregated-{date_object}.csv', index=False)